<a href="https://colab.research.google.com/github/LossHunter/AI/blob/main/Qwen3_OPRO_finetuning_30B_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab 셀 1: 라이브러리 설치
# !pip install -q "transformers==4.40.2" "datasets==2.18.0" "trl==0.8.6" "peft==0.10.0" "bitsandbytes==0.43.1"
!pip install -U transformers datasets trl peft bitsandbytes wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 91.3 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.19.11
    Uninstalling wandb-0.19.11:
      Successfully uninstalled wandb-0.19.11


In [ ]:
# prompt: unzip prepared_finetuning_dataset_splits_arrow_dpo
# 파일 명에 맞게 코드를 수정한다.
!unzip prepared_finetuning_dataset_splits_arrow_dpo

Archive:  prepared_finetuning_dataset_splits_arrow_dpo.zip
   creating: prepared_finetuning_dataset_splits_arrow_dpo/
  inflating: prepared_finetuning_dataset_splits_arrow_dpo/dataset_dict.json  
   creating: prepared_finetuning_dataset_splits_arrow_dpo/test/
  inflating: prepared_finetuning_dataset_splits_arrow_dpo/test/data-00000-of-00001.arrow  
  inflating: prepared_finetuning_dataset_splits_arrow_dpo/test/dataset_info.json  
  inflating: prepared_finetuning_dataset_splits_arrow_dpo/test/state.json  
   creating: prepared_finetuning_dataset_splits_arrow_dpo/train/
  inflating: prepared_finetuning_dataset_splits_arrow_dpo/train/data-00000-of-00001.arrow  
  inflating: prepared_finetuning_dataset_splits_arrow_dpo/train/dataset_info.json  
  inflating: prepared_finetuning_dataset_splits_arrow_dpo/train/state.json  
   creating: prepared_finetuning_dataset_splits_arrow_dpo/validation/
  inflating: prepared_finetuning_dataset_splits_arrow_dpo/validation/data-00000-of-00001.arrow  
  inf

In [ ]:
# Colab 셀 2: W&B 로그인
import wandb
# from google.colab import userdata

# W&B API 키를 입력하여 로그인
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: woosh98 (woosh98-inha-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# Colab 셀 2: 로컬 Arrow 데이터셋 불러오기
import datasets

# 데이터셋이 저장된 디렉토리 경로
# 압축 해제한 폴더 명을 수정한다.
dataset_path = "prepared_finetuning_dataset_splits_arrow_dpo"

# 디스크에서 DatasetDict 불러오기
try:
    dataset = datasets.load_from_disk(dataset_path)
    print("데이터셋을 성공적으로 불러왔습니다.")
    print(dataset)

    # 각 스플릿의 샘플 확인
    print("\nTrain 샘플:")
    print(dataset["train"][0])
    print("\nValidation 샘플:")
    print(dataset["validation"][0])

except FileNotFoundError:
    print(f"오류: '{dataset_path}' 경로를 찾을 수 없습니다.")
    print("Colab에 데이터셋 디렉토리를 올바르게 업로드했는지 확인해주세요.")
    # 예시 데이터셋으로 대체 (오류 발생 시)
    print("\n예시 데이터셋을 로드합니다.")
    dataset = datasets.load_dataset("mlabonne/orpo-dpo-mix-40k", split="train").train_test_split(test_size=0.05)
    # 가짜 validation 세트 만들기
    temp_split = dataset['test'].train_test_split(test_size=0.5)
    dataset['validation'] = temp_split['train']
    dataset['test'] = temp_split['test']
    print(dataset)

데이터셋을 성공적으로 불러왔습니다.
DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 610
    })
    validation: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 305
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 102
    })
})

Train 샘플:
{'prompt': '[과목명: 알고리즘설계 / 챕터: 1 / 문제유형: 객관식 / 요약: 이번 챕터에서는 알고리즘의 개념, 조건문과 반복문 등 ADL 구성 요소, 순환 알고리즘과 점화식, 시간·공간 복잡도 분석, 그리고 Big-O, Ω, Θ, o, ω 표기법을 활용한 점근적 분석 방법에 대해서 학습합니다.]\n아래 문제 중 더 나은 것은?', 'chosen': '질문: Omega notation (Ω)은 어떤 의미를 나타내는가?\n1) 상한\n2) 하한\n3) 중간값\n4) 함수값\n5) 실행 시간\n정답: 2\n해설: Omega notation은 점근적 하한을 나타낸다.', 'rejected': '질문: 다음 중 1.0 ms(밀리초)에 해당하는 단위는 무엇인가?  \n1) 1000 마이크로초(μs)  \n2) 0.001초  \n3) 10⁻³초  \n4) 1000 밀리초(ms)  \n5) 1000 나노초(ns)  \n정답: 1\n해설: 1 ms(밀리초)는 1000 마이크로초(μs)이다.'}

Validation 샘플:
{'prompt': '[과목명: 정보보호론 / 챕터: 1 / 문제유형: 객관식 / 요약: 이번 챕터에서는 컴퓨터 보안의 정의와 주요 자산의 가치, 위협·취약점·공격·통제의 관계, C-I-A 보안 목표, 접근 제어 및

In [ ]:
# Colab 셀 3: 모델 및 토크나이저 불러오기 (QLoRA 4비트 양자화 포함)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 사용할 Qwen3 모델 ID
# A100 GPU에서는 30B 모델을 충분히 학습할 수 있습니다.
model_id = "Qwen/Qwen3-4B"

# 4비트 양자화 설정
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# 모델 불러오기
# device_map="auto"는 accelerate 라이브러리가 최적의 GPU 장치를 자동으로 할당하도록 합니다.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.bfloat16, # A100/H100 GPU는 bfloat16을 지원하여 성능과 안정성을 높입니다.
    trust_remote_code=True
    attn_implementation="flash_attention_2"
)

# 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# 패딩 토큰 설정
# Qwen 모델은 기본적으로 pad_token이 없으므로, eos_token(문장 종료 토큰)을 pad_token으로 사용합니다.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Colab 셀 4: ORPO 학습 설정 (학습량 증가)
import os
import time
from trl import ORPOConfig
from peft import LoraConfig

# 1. W&B 프로젝트 이름 설정
os.environ["WANDB_PROJECT"] = f"qwen3-orpo-finetuning-{int(time.time())}"

# PEFT (LoRA) 설정
peft_config = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05,
    bias="none", task_type="CAUSAL_LM", target_modules="all-linear"
)

# 2. 실행 이름 설정
run_name = f"qwen3-30b-orpo-{int(time.time())}"

# ORPO 학습 인자 설정
orpo_args = ORPOConfig(
    output_dir=f"./results_{run_name}",
    run_name=run_name,
    bf16=True,
    learning_rate=5e-6,
    beta=0.1,
    lr_scheduler_type="cosine",
    max_length=8192,
    max_prompt_length=4096,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,

    # --- 이 부분을 수정합니다 ---
    # num_train_epochs=1, # 이 줄을 주석 처리하거나 삭제하고,
    max_steps=500,      # 원하는 총 스텝 수를 직접 지정합니다. (예: 500)
    # ---------------------------

    optim="paged_adamw_8bit",
    logging_steps=10,
    eval_steps=50,   # 평가 주기도 조절할 수 있습니다 (예: 50 스텝마다)
    save_steps=100,  # 저장 주기도 조절할 수 있습니다 (예: 100 스텝마다)
    remove_unused_columns=False,
    report_to="wandb",
)

In [ ]:
# Colab 셀 5: ORPO Trainer 생성 및 학습 시작 (최종 수정)
from trl import ORPOTrainer

# ORPOTrainer를 생성합니다.
# tokenizer 객체를 'processing_class' 인수로 전달하는 것이 핵심입니다.
trainer = ORPOTrainer(
    model=model,
    args=orpo_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    processing_class=tokenizer,  # <--- 이 부분이 수정되었습니다.
)

# 학습 시작
# 이제 오류 없이 W&B 로깅과 함께 학습이 진행됩니다.
trainer.train()

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,8.010400
20,7.559800
30,7.174800
40,6.660200
50,6.310300
60,6.058900
70,5.662200
80,4.792900
90,4.971600
100,4.593600


TrainOutput(global_step=500, training_loss=3.367634449005127, metrics={'train_runtime': 1584.8788, 'train_samples_per_second': 2.524, 'train_steps_per_second': 0.315, 'total_flos': 0.0, 'train_loss': 3.367634449005127, 'epoch': 6.498360655737705})

In [ ]:
# Colab 셀 6: 모델 저장 및 W&B 종료
final_model_path = f"./{run_name}-final"
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)

print(f"✅ 학습된 모델이 '{final_model_path}' 경로에 저장되었습니다.")

# W&B 실행 종료
wandb.finish()

✅ 학습된 모델이 './qwen3-4b-orpo-1749985136-final' 경로에 저장되었습니다.


train/epoch,▁▁▁▂▂▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/grad_norm,▄▄▂▂▂▁▄▄▁▁▃▂▂▃▂▅▃▄▃▂▄▅▄▅▅▅▄▄▅▄█▅▄▅▄▅▅▄▅▄
train/learning_rate,█▄▃▁█████▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁▁▁▁
train/log_odds_chosen,▁▁▁▁▂▁▁▁▂▂▂▂▂▃▃▃▄▄▄▅▅▆▅▇▆▆▆▇▇▇▇▇▇▇██▇▇█▇
train/log_odds_ratio,▁▁▂▂▂▁▁▂▂▂▃▃▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████▇███
train/logits/chosen,▂▂▂▂▃▁▂▂▃▃▃▃▄▄▅▆▆▆▆▆▇▇▇▇▇▇▇█▇███████████
train/logits/rejected,▁▂▂▂▂▁▁▂▂▂▃▂▃▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇█▇██▇▇█▇█▇
train/logps/chosen,▁▁▂▂▂▁▁▂▃▄▅▅▅▆▇▇▇▇▇▇▇▇█▇▇▇██▇█▇███▇█████
train/logps/rejected,▇██▇█▇▇██▇██████▇▇▆▆▅▅▅▅▃▃▃▄▃▃▂▂▃▂▂▁▁▁▁▂
train/loss,██▇▆▆▇▇▆▆▆▄▅▄▄▄▃▃▃▂▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁


In [ ]:
# Colab 셀 1: 허깅페이스 Hub 로그인
from huggingface_hub import notebook_login

# 이 셀을 실행하면 텍스트 박스가 나타납니다.
# 위 링크에서 발급받은 허깅페이스 토큰을 복사하여 붙여넣고 로그인하세요.
notebook_login()

In [ ]:
# Colab 셀 2: 업로드 정보 및 경로 설정
import torch

# ----------------- (수정 필요) -----------------
# 허깅페이스에 생성할 리포지토리 ID를 지정합니다.
# "totalcream/{원하는 모델 이름}" 형식으로 작성해주세요.
hf_repo_id = "totalcream/Qwen3-ORPO-finetuning-30B"
# ----------------------------------------------

# 이전에 학습한 결과물이 저장된 경로들
base_model_id = "Qwen/Qwen3-30B" # 원본 모델
adapter_path = "/content/qwen3-30b-orpo-1749985136-final" # 이전 단계에서 저장한 어댑터 경로

print(f"✅ 허깅페이스 리포지토리: https://huggingface.co/{hf_repo_id}")
print(f"✅ 베이스 모델: {base_model_id}")
print(f"✅ 로컬 어댑터 경로: {adapter_path}")

✅ 허깅페이스 리포지토리: https://huggingface.co/totalcream/Qwen3-ORPO-finetuning
✅ 베이스 모델: Qwen/Qwen3-4B
✅ 로컬 어댑터 경로: /content/qwen3-4b-orpo-1749985136-final


In [ ]:
# Colab 셀 3: 어댑터(Adapter)만 Hub에 업로드
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

print(f"1. '{adapter_path}' 경로의 어댑터를 로딩합니다...")
# PeftModel.from_pretrained는 어댑터만 불러오는 것이 아니라,
# 베이스 모델 위에 어댑터를 얹은(Peft) 모델 객체를 생성합니다.
# 아직 병합된 상태는 아닙니다.
model = PeftModel.from_pretrained(
    AutoModelForCausalLM.from_pretrained(
        base_model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True
    ),
    adapter_path
)
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)


print(f"2. 어댑터를 '{hf_repo_id}' 리포지토리에 업로드합니다...")
# push_to_hub를 사용하면 LoRA 어댑터 관련 파일들(adapter_*.safetensors, adapter_config.json)이 업로드됩니다.
model.push_to_hub(hf_repo_id, safe_serialization=True)
tokenizer.push_to_hub(hf_repo_id) # 토크나이저도 함께 업로드합니다.

print(f"✅ 어댑터 업로드 완료! -> https://huggingface.co/{hf_repo_id}")

1. '/content/qwen3-4b-orpo-1749985136-final' 경로의 어댑터를 로딩합니다...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

2. 어댑터를 'totalcream/Qwen3-ORPO-finetuning' 리포지토리에 업로드합니다...


adapter_model.safetensors:   0%|          | 0.00/132M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

✅ 어댑터 업로드 완료! -> https://huggingface.co/totalcream/Qwen3-ORPO-finetuning


In [ ]:
# Colab 셀 4: 모델 병합 후 전체 모델(Merged Model) 업로드

print("3. 어댑터를 베이스 모델에 병합합니다...")
# merge_and_unload() 함수는 어댑터 레이어를 베이스 모델에 합친 후,
# 더 이상 필요 없어진 Peft 레이어를 메모리에서 해제합니다.
# 반환되는 것은 순수한 Transformers 모델 객체입니다.
merged_model = model.merge_and_unload()
print("병합 완료!")

print(f"4. 병합된 전체 모델을 '{hf_repo_id}'에 업로드합니다...")
# 이제 병합된 모델 객체의 push_to_hub를 호출합니다.
# 이렇게 하면 전체 모델 가중치 파일(safetensors)이 리포지토리에 추가로 업로드됩니다.
merged_model.push_to_hub(hf_repo_id, safe_serialization=True)

print(f"✅ 병합된 전체 모델 업로드 완료! -> https://huggingface.co/{hf_repo_id}")

3. 어댑터를 베이스 모델에 병합합니다...
병합 완료!
4. 병합된 전체 모델을 'totalcream/Qwen3-ORPO-finetuning'에 업로드합니다...


model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

✅ 병합된 전체 모델 업로드 완료! -> https://huggingface.co/totalcream/Qwen3-ORPO-finetuning


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# 당신이 Hub에 올린 모델 ID
my_model_id = "totalcream/Qwen3-ORPO-finetuning"

# 다른 사용자가 당신의 '병합된 모델'을 4비트로 불러와 추론하는 코드
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    my_model_id,
    quantization_config=quantization_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(my_model_id)

# 이제 이 모델로 효율적인 4비트 추론을 할 수 있습니다.